In [9]:
import pandas as pd
import os
import numpy as np
import librosa.display
import IPython
import matplotlib.pyplot as plt
import librosa
from scipy.io.wavfile import read as read_wav
import seaborn as sns
import torchaudio
import librosa
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch
import torch.nn as nn

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import librosa
import soundfile as sf

import pandas as pd
import os
import numpy as np
import librosa.display
import IPython
import matplotlib.pyplot as plt
import librosa
from scipy.io.wavfile import read as read_wav
import seaborn as sns
import torchaudio


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def load_data_to_df(data_dir='shords_dataset/'):
    data = []
    for cl in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, cl)
        for file in os.listdir(class_dir):
            if file.endswith('wav'):
                file_path = os.path.join(class_dir, file)
                data.append({'file_path': file_path, 'class': cl})
    df = pd.DataFrame(data)
    return df


In [4]:
data = load_data_to_df()
data.head()

,file_path,class
0,shords_dataset/major/4_21.wav,major
1,shords_dataset/major/1_8.wav,major
2,shords_dataset/major/9_49.wav,major
3,shords_dataset/major/9_61.wav,major
4,shords_dataset/major/9_75.wav,major


In [6]:
major_df = data[data['class'] == 'major']


In [25]:
def load_and_convert_to_spectrogram(file_path, n_fft=2048, hop_length=512, n_mels=128, pad_mode='constant', pad_value=-80.0):
    audio, sr = librosa.load(file_path, sr=None)
    spectrogram = librosa.feature.melspectrogram(y = audio, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    spectrogram = librosa.power_to_db(spectrogram, ref=np.max)  # Convert to dB scale
    
    # Add padding to make all spectrograms the same size
    max_length = 400  # Adjust this value as needed based on your data
    if spectrogram.shape[1] < max_length:
        pad_width = max_length - spectrogram.shape[1]
        spectrogram = np.pad(spectrogram, pad_width=((0, 0), (0, pad_width)), mode=pad_mode, constant_values=pad_value)
    else:
        spectrogram = spectrogram[:, :max_length]
    
    return spectrogram

class AudioDataset(Dataset):
    def __init__(self, df, n_fft=2048, hop_length=512, n_mels=128):
        self.df = df
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        file_path = self.df.iloc[idx]['file_path']
        
        # Load and convert audio to spectrogram
        spectrogram = load_and_convert_to_spectrogram(file_path, self.n_fft, self.hop_length, self.n_mels)
        
        # Convert spectrogram to PyTorch tensor
        spectrogram = torch.FloatTensor(spectrogram)  # Convert to float tensor
        
        # Add channel dimension (assuming you're using CNNs)
        spectrogram = spectrogram.unsqueeze(0)  # Shape: (1, n_mels, T), where T is time steps
        
        return spectrogram

# Initialize dataset and dataloader
dataset = AudioDataset(major_df)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Example of iterating through the data loader
for batch in data_loader:
    spectrogram = batch
    print(f"Spectrogram shape: {spectrogram.shape}")

Spectrogram shape: torch.Size([32, 1, 128, 400])
Spectrogram shape: torch.Size([32, 1, 128, 400])
Spectrogram shape: torch.Size([32, 1, 128, 400])
Spectrogram shape: torch.Size([32, 1, 128, 400])
Spectrogram shape: torch.Size([32, 1, 128, 400])
Spectrogram shape: torch.Size([32, 1, 128, 400])
Spectrogram shape: torch.Size([32, 1, 128, 400])
Spectrogram shape: torch.Size([26, 1, 128, 400])


In [37]:
import itertools

# Define Generator and Discriminator architectures
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(input_dim[0], 64, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(512),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(512, 1024, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(1024),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(1024, 512, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(512),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(128, output_dim[0], kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )
    
    def forward(self, x):
        return self.model(x)


class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(input_dim[0], 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(256, 512, kernel_size=4, stride=1, padding=1),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)
        )
    
    def forward(self, x):
        return self.model(x)

In [38]:
class CycleGAN(nn.Module):
    def __init__(self, G_AB, G_BA, D_A, D_B):
        super(CycleGAN, self).__init__()
        self.G_AB = G_AB  # Generator: Domain A to B
        self.G_BA = G_BA  # Generator: Domain B to A
        self.D_A = D_A    # Discriminator: Domain A
        self.D_B = D_B    # Discriminator: Domain B
    
    def forward(self, x_A, x_B):
        # Forward pass through generators
        x_BA = self.G_BA(x_B)  # Generate B from A
        x_AB = self.G_AB(x_A)  # Generate A from B
        
        # Forward pass through discriminators (for adversarial loss)
        pred_A = self.D_A(x_A)  # Discriminate real A
        pred_BA = self.D_A(x_BA)  # Discriminate generated A (from B)
        
        pred_B = self.D_B(x_B)  # Discriminate real B
        pred_AB = self.D_B(x_AB)  # Discriminate generated B (from A)
        
        return x_BA, x_AB, pred_A, pred_BA, pred_B, pred_AB


In [41]:
input_dim = (1, 128, 400)
output_dim = (1, 128, 400)

criterion_GAN = nn.MSELoss()  # Mean Squared Error loss for GAN loss
criterion_cycle = nn.L1Loss()  # L1 loss for cycle consistency

# Initialize generators and discriminators
G_AB = Generator(input_dim, output_dim)
G_BA = Generator(input_dim, output_dim)
D_A = Discriminator(input_dim)
D_B = Discriminator(input_dim)

# Initialize CycleGAN model
cycle_gan = CycleGAN(G_AB, G_BA, D_A, D_B)

# Optimizers
optimizer_G = torch.optim.Adam(itertools.chain(G_AB.parameters(), G_BA.parameters()), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Print model summary (optional)
print(G_AB)
print(D_A)


Generator(
  (model): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (5): ReLU(inplace=True)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (8): ReLU(inplace=True)
    (9): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (10): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (11): ReLU(inplace=True)
    (12): Conv2d(512, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (13): InstanceNorm2d(1024, eps=1e-05, momentum=0.1, affine=False, track_running_stats=

In [44]:
device = 'mps'
num_epochs = 1000

for epoch in range(num_epochs):
    for i, batch in enumerate(data_loader):
        real_A = batch['A']  # Assuming 'A' corresponds to real_A in your dataset
        real_B = batch['B']  # Assuming 'B' corresponds to real_B in your dataset
        
        # Move data to device if using GPU
        real_A = real_A.to(device)
        real_B = real_B.to(device)
        
        # Train Discriminator A
        optimizer_D_A.zero_grad()
        
        with torch.no_grad():
            fake_B = cycle_gan.G_AB(real_A)
        pred_real_A = cycle_gan.D_A(real_A)
        pred_fake_B = cycle_gan.D_A(fake_B.detach())
        
        loss_D_A_real = criterion_GAN(pred_real_A, torch.ones_like(pred_real_A))
        loss_D_A_fake = criterion_GAN(pred_fake_B, torch.zeros_like(pred_fake_B))
        
        loss_D_A = 0.5 * (loss_D_A_real + loss_D_A_fake)
        loss_D_A.backward()
        optimizer_D_A.step()
        
        # Train Discriminator B
        optimizer_D_B.zero_grad()
        
        with torch.no_grad():
            fake_A = cycle_gan.G_BA(real_B)
        pred_real_B = cycle_gan.D_B(real_B)
        pred_fake_A = cycle_gan.D_B(fake_A.detach())
        
        loss_D_B_real = criterion_GAN(pred_real_B, torch.ones_like(pred_real_B))
        loss_D_B_fake = criterion_GAN(pred_fake_A, torch.zeros_like(pred_fake_A))
        
        loss_D_B = 0.5 * (loss_D_B_real + loss_D_B_fake)
        loss_D_B.backward()
        optimizer_D_B.step()
        
        # Train Generators
        optimizer_G.zero_grad()
        
        # Adversarial loss
        pred_fake_A = cycle_gan.D_B(fake_A)
        pred_fake_B = cycle_gan.D_A(fake_B)
        
        loss_GAN_AB = criterion_GAN(pred_fake_A, torch.ones_like(pred_fake_A))
        loss_GAN_BA = criterion_GAN(pred_fake_B, torch.ones_like(pred_fake_B))
        
        # Cycle-consistency loss
        recov_A = cycle_gan.G_BA(fake_B)
        recov_B = cycle_gan.G_AB(fake_A)
        
        loss_cycle_A = criterion_cycle(recov_A, real_A)
        loss_cycle_B = criterion_cycle(recov_B, real_B)
        
        # Total generator loss
        loss_G = loss_GAN_AB + loss_GAN_BA + 10 * (loss_cycle_A + loss_cycle_B)
        loss_G.backward()
        optimizer_G.step()
        
        # Print losses
        if i % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}], Step [{i}/{len(data_loader)}], "
                  f"Loss_G: {loss_G.item():.4f}, Loss_D_A: {loss_D_A.item():.4f}, Loss_D_B: {loss_D_B.item():.4f}")

TypeError: new(): invalid data type 'str'